In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)
torch.cuda.current_device()

1

# Download YT video

In [7]:
from pytube import YouTube

video_url = "https://www.youtube.com/watch?v=Pt9LelJ0fL0"

# Create a YouTube object
yt = YouTube(video_url)

# Get the highest quality stream available
stream = yt.streams.get_highest_resolution()

# Download the video to the current working directory
stream.download(filename="demo_video.mp4")

print("Video downloaded successfully!")

Video downloaded successfully!


# Convert MP4 to MP3

In [10]:
from moviepy.editor import *

video = VideoFileClip("demo_video.mp4")
audio = video.audio

audio.write_audiofile("demo_audio.mp3")

print("Audio extracted and saved as MP3!")

MoviePy - Writing audio in demo_audio.mp3


MoviePy - Done.
Audio extracted and saved as MP3!


# Transcribe audio

In [11]:
import whisper
model = whisper.load_model("medium")
# move to gpu
model.to(device)
options = whisper.DecodingOptions(task="translate", language="en")

result = whisper.transcribe(model, audio="demo_audio.mp3", language="en", task="translate")


In [13]:
result['text']

" In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's start with cloud computing. Cloud computing is a delivery model for services, services like storage services. Because whenever you are storing your unstructured or structured data in files and databases, cloud delivers you dozens of services to do so. You pretty much can choose any technology that is popular in the market and you will be able to use it within your Azure environment. In this case, storage doesn't only mean storing data, but also all 

In [14]:
len(result['text'].split())

1570

In [21]:
with open('course_data.txt', 'w') as file:
    file.write(result['text'])

In [ ]:
!pip install "unstructured[local-inference]"
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
!pip install layoutparser[layoutmodels,tesseract]

In [29]:
from langchain.document_loaders import UnstructuredFileLoader
loader = UnstructuredFileLoader("course_data.txt")
docs = loader.load()
docs

[Document(page_content="In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's start with cloud computing. Cloud computing is a delivery model for services, services like storage services. Because whenever you are storing your unstructured or structured data in files and databases, cloud delivers you dozens of services to do so. You pretty much can choose any technology that is popular in the market and you will be able to use it within your Azure environment. In this case, storage doesn't only mean stori

In [43]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [ ]:
!pip install chromadb

# Conversation Agent demo

In [17]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [51]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2, temperature=0.8,  top_p=1,)

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [52]:
course_instruction = f"""Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information: {result['text']}"""


# prompt.format(instruction=course_instruction)

In [53]:
from langchain import PromptTemplate, LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.run(course_instruction)

" Congratulations! You have successfully generated a mini-course containing 3 lessons and an upsell offer based on the information provided. The first episode discussed cloud computing and the most important vocabulary to understand when working with cloud. The objectives of the first episode included learning what cloud computing is, its benefits, and the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance, and disaster recovery. The episode also provided a summary of each of these terms and discussed the availability and scalability of cloud services. To test your knowledge, visit marchag.io/az900 to access the materials and cheat sheet. Thanks again for watching and don't forget to like, comment, share, and subscribe!"

In [63]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
window_memory = ConversationBufferWindowMemory(k=20)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=window_memory
)

In [64]:
conversation.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [65]:
conversation.predict(input=course_instruction)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information:  In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's 

' Hi Human! It sounds like you want me to create a mini-course containing 3 lessons and an "upsell" offer at the end based on cloud computing. Is that correct?'

In [66]:
conversation.predict(input="What capabilities do students have before they take this mini-course?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information:  In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's s

' Before taking this mini-course, students should understand the basics of cloud computing, including key vocabulary like high availability, scalability, elasticity, agility, fault tolerance, and disaster recovery. They should also be familiar with the services cloud computing offers, such as storage services and compute-powered resources. Additionally, it would be beneficial to know about the processes of scaling vertically and horizontally, as well as the concept of elasticity and agility. Finally, students should have some understanding of fault tolerance and disaster recovery.'

In [67]:
conversation.predict(input="What capabilities do students have after they take this mini-course?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information:  In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's s

' After taking this mini-course, students should have a better understanding of cloud computing and the services it offers, including storage services and compute-powered resources. They should also know more about the process of scaling vertically and horizontally, as well as elasticity, agility, fault tolerance, and disaster recovery. Additionally, they should be able to recognize the difference between cloud computing and on-premises services, as well as the importance of availability and disaster recovery.'

In [68]:
conversation.predict(input="Generate a 3 multiple-choice quiz questions on what was covered")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information:  In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let's look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let's s

' \n\nQuestion 1: Which of the following is a characteristic of cloud computing? \na. Scalability\nb. Agility\nc. Elasticity\nd. All of the above\n\nQuestion 2: What is the main difference between cloud computing and on-premises services?\na. Cloud computing services are faster to access than on-premises services\nb. Cloud computing services are more expensive than on-premises services\nc. Cloud computing services can be accessed remotely, while on-premises services cannot\nd. Cloud computing services are less secure than on-premises services\n\nQuestion 3: What is fault tolerance?\na. The ability of the system to remain up and running during component and service failures\nb. The ability of the system to scale dynamically\nc. The ability of the system to recover from events that have taken down an entire region or service\nd. The ability of the system to remain available to users or other systems'

In [76]:
print(conversation.memory.buffer)

[HumanMessage(content='Generate a produced mini-course containing 3 lessons and an "upsell" offer at the end, based on the following information:  In the first episode we will be talking about what cloud computing is, but also what is the most important vocabulary that you should understand when working with cloud in general. It both describes what cloud computing is, but also what are its benefits. Stay tuned! Let\'s look at the objectives of the first episode. Today we will learn what cloud computing is, but also what are its benefits by learning the most important vocabulary like high availability, scalability, elasticity, agility, fault tolerance and also disaster recovery. Let\'s start with cloud computing. Cloud computing is a delivery model for services, services like storage services. Because whenever you are storing your unstructured or structured data in files and databases, cloud delivers you dozens of services to do so. You pretty much can choose any technology that is popu

# Text to Speech

In [ ]:
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets

In [ ]:
!pip install soundfile

In [77]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
my_text = 'After taking this mini-course, students should have a better understanding of cloud computing and the services it offers, including storage services and compute-powered resources. They should also know more about the process of scaling vertically and horizontally, as well as elasticity, agility, fault tolerance, and disaster recovery. Additionally, they should be able to recognize the difference between cloud computing and on-premises services, as well as the importance of availability and disaster recovery.'
inputs = processor(text=my_text, return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

Found cached dataset cmu-arctic-xvectors (C:/Users/m1000/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)
